### Wilcoxon

In [381]:
import os
import numpy as np
import pandas as pd
import time
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
import os
from scipy.stats import wilcoxon
os.getcwd()
os.chdir('C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask')
os.getcwd()

'C:\\Users\\ASUS\\Desktop\\Github\\Face_Recognition_with_mask'

In [382]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']
# Test_dataset_50id_vector_Non_masked.txt
# Test_dataset_50id_vector_Masked_Ver_2.txt
file = open("C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask/Data/Test_dataset_50id_vector_Masked_Ver_2.txt",'r')
lines = file.readlines()
first = True
df = None
count = 1
vector_list = []
gender_list = []
race_list = []
age_list = []
id_list = []
for line in lines:
    data = line.split('|')
    image_id = line.split('/')[3]
    image_id = image_id[:image_id.find('\\')]
    id_list.append(image_id)
    list_of_vector = data[1].split(' ')
    list_of_vector = map(float, list_of_vector)
    list_of_vector = list(list_of_vector)
    vector_list.append(list_of_vector)
    list_age = data[2].split(' ')
    list_gender = data[3].split(' ')
    list_race = data[4].split(' ') 
    if np.argmax(list_gender) == 0:
        gender_list.append('Women')
    elif np.argmax(list_gender) == 1:
        gender_list.append('Men')   
    if np.argmax(list_race) == 0:
        race_list.append('Asian')
    elif np.argmax(list_race) == 1:
        race_list.append('Indian')
    elif np.argmax(list_race) == 2:
        race_list.append('Black')
    elif np.argmax(list_race) == 3:
        race_list.append('White')
    elif np.argmax(list_race) == 4:
        race_list.append('Middle_Eastern')
    elif np.argmax(list_race) == 5:
        race_list.append('Latino_Hispanic') 
    list_age = map(float, list_age)
    list_age = list(list_age)
    age_predictions = np.array(list_age)
    output_indexes = np.array([i for i in range(0, 101)])
    apparent_age = np.sum(age_predictions * output_indexes)
    apparent_age = round(apparent_age)
    if apparent_age >= 0 and apparent_age <= 9:
        age_list.append('0-9')
    elif apparent_age >= 10 and apparent_age <= 19:
        age_list.append('10-19')
    elif apparent_age >= 20 and apparent_age <= 29:
        age_list.append('20-29')
    elif apparent_age >= 30 and apparent_age <= 39:
        age_list.append('30-39')
    elif apparent_age >= 40 and apparent_age <= 49:
        age_list.append('40-49')
    elif apparent_age >= 50 and apparent_age <= 59:
        age_list.append('50-59')
    elif apparent_age >= 60 and apparent_age <= 69:
        age_list.append('60-69')
    elif apparent_age >= 70 and apparent_age <= 79:
        age_list.append('70-79')
    elif apparent_age >= 80 and apparent_age <= 89:                   
        age_list.append('80-89')
    elif apparent_age >= 90 and apparent_age <= 100:
        age_list.append('90-100')

dataframe_test = pd.DataFrame(vector_list)
one_hot_gender = pd.get_dummies(gender_list)
one_hot_race = pd.get_dummies(race_list)
one_hot_age = pd.get_dummies(age_list)

for i in range(10):
    if age_label_list[i] not in one_hot_age.columns:
        one_hot_age.insert(loc=i,column=age_label_list[i],value = 0)

# print(id_list)
le = preprocessing.LabelEncoder()
le.fit(id_list)
encoded_id_test = le.transform(id_list)        

dataframe_test = dataframe_test.join(one_hot_age)
dataframe_test = dataframe_test.join(one_hot_gender)
dataframe_test = dataframe_test.join(one_hot_race)
dataframe_test['id'] = encoded_id_test
dataframe_test

,0,1,2,3,4,5,6,7,8,9,...,90-100,Men,Women,Asian,Black,Indian,Latino_Hispanic,Middle_Eastern,White,id
0,-0.008172,-0.006911,-0.015271,0.008623,0.000918,0.000767,-0.002832,0.014805,-0.003724,0.006555,...,0,1,0,0,0,1,0,0,0,0
1,0.006298,-0.009886,-0.005260,0.016638,0.008184,0.003435,0.001660,0.013513,-0.007286,0.007281,...,0,0,1,0,0,0,0,0,1,0
2,-0.003654,-0.005899,-0.011440,0.009473,-0.000083,-0.002358,-0.006520,0.013124,-0.009252,0.009202,...,0,0,1,0,0,0,0,0,1,0
3,0.007064,-0.004087,-0.017468,0.015006,-0.005484,0.003320,0.010896,0.019482,-0.004352,0.004539,...,0,1,0,0,0,0,0,0,1,0
4,0.010283,0.009022,0.001387,0.004486,0.005245,0.016129,0.012244,0.022016,0.002608,0.009988,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,0.014094,-0.004892,-0.010881,0.012747,0.020536,0.003168,-0.005631,0.026119,-0.006610,-0.003373,...,0,0,1,0,0,0,0,0,1,49
2119,0.009694,-0.006320,-0.008706,0.020135,0.020386,0.009901,-0.002650,0.024439,-0.002204,0.008890,...,0,0,1,0,0,0,0,0,1,49
2120,0.021144,-0.002316,-0.004799,0.014893,0.027049,0.016815,-0.002363,0.027073,0.010423,0.017861,...,0,0,1,0,0,0,0,0,1,49
2121,0.016763,-0.008999,-0.016320,0.019693,0.016492,0.017646,-0.004644,0.021110,0.000492,0.001589,...,0,0,1,0,0,0,0,0,1,49


In [383]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']
gender_label_list = ['Men','Women']
race_label_list = ['Asian','Black','Indian','Latino_Hispanic','Middle_Eastern','White']

all_features_label_list = age_label_list + gender_label_list + race_label_list
age_gender_label = race_label_list
age_race_label = gender_label_list
gender_race_label = age_label_list
age_label = gender_label_list + race_label_list
gender_label = age_label_list + race_label_list
race_label = age_label_list + gender_label_list

all_feature_activate = True
if all_feature_activate is not True:
    dataframe_test = dataframe_test.drop(gender_race_label, axis=1)
dataframe_test

,0,1,2,3,4,5,6,7,8,9,...,90-100,Men,Women,Asian,Black,Indian,Latino_Hispanic,Middle_Eastern,White,id
0,-0.008172,-0.006911,-0.015271,0.008623,0.000918,0.000767,-0.002832,0.014805,-0.003724,0.006555,...,0,1,0,0,0,1,0,0,0,0
1,0.006298,-0.009886,-0.005260,0.016638,0.008184,0.003435,0.001660,0.013513,-0.007286,0.007281,...,0,0,1,0,0,0,0,0,1,0
2,-0.003654,-0.005899,-0.011440,0.009473,-0.000083,-0.002358,-0.006520,0.013124,-0.009252,0.009202,...,0,0,1,0,0,0,0,0,1,0
3,0.007064,-0.004087,-0.017468,0.015006,-0.005484,0.003320,0.010896,0.019482,-0.004352,0.004539,...,0,1,0,0,0,0,0,0,1,0
4,0.010283,0.009022,0.001387,0.004486,0.005245,0.016129,0.012244,0.022016,0.002608,0.009988,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,0.014094,-0.004892,-0.010881,0.012747,0.020536,0.003168,-0.005631,0.026119,-0.006610,-0.003373,...,0,0,1,0,0,0,0,0,1,49
2119,0.009694,-0.006320,-0.008706,0.020135,0.020386,0.009901,-0.002650,0.024439,-0.002204,0.008890,...,0,0,1,0,0,0,0,0,1,49
2120,0.021144,-0.002316,-0.004799,0.014893,0.027049,0.016815,-0.002363,0.027073,0.010423,0.017861,...,0,0,1,0,0,0,0,0,1,49
2121,0.016763,-0.008999,-0.016320,0.019693,0.016492,0.017646,-0.004644,0.021110,0.000492,0.001589,...,0,0,1,0,0,0,0,0,1,49


In [384]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']

file = open("C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask/Data/Test_dataset_50id_vector_Masked_Ver_2.txt",'r')
lines = file.readlines()
first = True
df = None
count = 1
vector_list = []
gender_list = []
race_list = []
age_list = []
id_list = []
for line in lines:
    data = line.split('|')
    image_id = line.split('/')[3]
    image_id = image_id[:image_id.find('\\')]
    id_list.append(image_id)
    list_of_vector = data[1].split(' ')
    list_of_vector = map(float, list_of_vector)
    list_of_vector = list(list_of_vector)
    vector_list.append(list_of_vector)
    list_age = data[2].split(' ')
    list_gender = data[3].split(' ')
    list_race = data[4].split(' ') 
    if np.argmax(list_gender) == 0:
        gender_list.append('Women')
    elif np.argmax(list_gender) == 1:
        gender_list.append('Men')   
    if np.argmax(list_race) == 0:
        race_list.append('Asian')
    elif np.argmax(list_race) == 1:
        race_list.append('Indian')
    elif np.argmax(list_race) == 2:
        race_list.append('Black')
    elif np.argmax(list_race) == 3:
        race_list.append('White')
    elif np.argmax(list_race) == 4:
        race_list.append('Middle_Eastern')
    elif np.argmax(list_race) == 5:
        race_list.append('Latino_Hispanic') 
    list_age = map(float, list_age)
    list_age = list(list_age)
    age_predictions = np.array(list_age)
    output_indexes = np.array([i for i in range(0, 101)])
    apparent_age = np.sum(age_predictions * output_indexes)
    apparent_age = round(apparent_age)
    if apparent_age >= 0 and apparent_age <= 9:
        age_list.append('0-9')
    elif apparent_age >= 10 and apparent_age <= 19:
        age_list.append('10-19')
    elif apparent_age >= 20 and apparent_age <= 29:
        age_list.append('20-29')
    elif apparent_age >= 30 and apparent_age <= 39:
        age_list.append('30-39')
    elif apparent_age >= 40 and apparent_age <= 49:
        age_list.append('40-49')
    elif apparent_age >= 50 and apparent_age <= 59:
        age_list.append('50-59')
    elif apparent_age >= 60 and apparent_age <= 69:
        age_list.append('60-69')
    elif apparent_age >= 70 and apparent_age <= 79:
        age_list.append('70-79')
    elif apparent_age >= 80 and apparent_age <= 89:                   
        age_list.append('80-89')
    elif apparent_age >= 90 and apparent_age <= 100:
        age_list.append('90-100')

dataframe_test_2 = pd.DataFrame(vector_list)
one_hot_gender = pd.get_dummies(gender_list)
one_hot_race = pd.get_dummies(race_list)
one_hot_age = pd.get_dummies(age_list)

for i in range(10):
    if age_label_list[i] not in one_hot_age.columns:
        one_hot_age.insert(loc=i,column=age_label_list[i],value = 0)

le = preprocessing.LabelEncoder()
le.fit(id_list)
encoded_id_test_2 = le.transform(id_list)        

dataframe_test_2 = dataframe_test_2.join(one_hot_age)
dataframe_test_2 = dataframe_test_2.join(one_hot_gender)
dataframe_test_2 = dataframe_test_2.join(one_hot_race)
dataframe_test_2['id'] = encoded_id_test_2
dataframe_test_2

,0,1,2,3,4,5,6,7,8,9,...,90-100,Men,Women,Asian,Black,Indian,Latino_Hispanic,Middle_Eastern,White,id
0,-0.008172,-0.006911,-0.015271,0.008623,0.000918,0.000767,-0.002832,0.014805,-0.003724,0.006555,...,0,1,0,0,0,1,0,0,0,0
1,0.006298,-0.009886,-0.005260,0.016638,0.008184,0.003435,0.001660,0.013513,-0.007286,0.007281,...,0,0,1,0,0,0,0,0,1,0
2,-0.003654,-0.005899,-0.011440,0.009473,-0.000083,-0.002358,-0.006520,0.013124,-0.009252,0.009202,...,0,0,1,0,0,0,0,0,1,0
3,0.007064,-0.004087,-0.017468,0.015006,-0.005484,0.003320,0.010896,0.019482,-0.004352,0.004539,...,0,1,0,0,0,0,0,0,1,0
4,0.010283,0.009022,0.001387,0.004486,0.005245,0.016129,0.012244,0.022016,0.002608,0.009988,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,0.014094,-0.004892,-0.010881,0.012747,0.020536,0.003168,-0.005631,0.026119,-0.006610,-0.003373,...,0,0,1,0,0,0,0,0,1,49
2119,0.009694,-0.006320,-0.008706,0.020135,0.020386,0.009901,-0.002650,0.024439,-0.002204,0.008890,...,0,0,1,0,0,0,0,0,1,49
2120,0.021144,-0.002316,-0.004799,0.014893,0.027049,0.016815,-0.002363,0.027073,0.010423,0.017861,...,0,0,1,0,0,0,0,0,1,49
2121,0.016763,-0.008999,-0.016320,0.019693,0.016492,0.017646,-0.004644,0.021110,0.000492,0.001589,...,0,0,1,0,0,0,0,0,1,49


In [385]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']
gender_label_list = ['Men','Women']
race_label_list = ['Asian','Black','Indian','Latino_Hispanic','Middle_Eastern','White']

all_features_label_list = age_label_list + gender_label_list + race_label_list
age_gender_label = race_label_list
age_race_label = gender_label_list
gender_race_label = age_label_list
age_label = gender_label_list + race_label_list
gender_label = age_label_list + race_label_list
race_label = age_label_list + gender_label_list

all_feature_activate = False
if all_feature_activate is not True:
    dataframe_test_2 = dataframe_test_2.drop(all_features_label_list, axis=1)

In [386]:
dataframe_test

,0,1,2,3,4,5,6,7,8,9,...,90-100,Men,Women,Asian,Black,Indian,Latino_Hispanic,Middle_Eastern,White,id
0,-0.008172,-0.006911,-0.015271,0.008623,0.000918,0.000767,-0.002832,0.014805,-0.003724,0.006555,...,0,1,0,0,0,1,0,0,0,0
1,0.006298,-0.009886,-0.005260,0.016638,0.008184,0.003435,0.001660,0.013513,-0.007286,0.007281,...,0,0,1,0,0,0,0,0,1,0
2,-0.003654,-0.005899,-0.011440,0.009473,-0.000083,-0.002358,-0.006520,0.013124,-0.009252,0.009202,...,0,0,1,0,0,0,0,0,1,0
3,0.007064,-0.004087,-0.017468,0.015006,-0.005484,0.003320,0.010896,0.019482,-0.004352,0.004539,...,0,1,0,0,0,0,0,0,1,0
4,0.010283,0.009022,0.001387,0.004486,0.005245,0.016129,0.012244,0.022016,0.002608,0.009988,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,0.014094,-0.004892,-0.010881,0.012747,0.020536,0.003168,-0.005631,0.026119,-0.006610,-0.003373,...,0,0,1,0,0,0,0,0,1,49
2119,0.009694,-0.006320,-0.008706,0.020135,0.020386,0.009901,-0.002650,0.024439,-0.002204,0.008890,...,0,0,1,0,0,0,0,0,1,49
2120,0.021144,-0.002316,-0.004799,0.014893,0.027049,0.016815,-0.002363,0.027073,0.010423,0.017861,...,0,0,1,0,0,0,0,0,1,49
2121,0.016763,-0.008999,-0.016320,0.019693,0.016492,0.017646,-0.004644,0.021110,0.000492,0.001589,...,0,0,1,0,0,0,0,0,1,49


In [387]:
print(dataframe_test['id'].values)
id = dataframe_test['id'].values.tolist()
dataframe_test = dataframe_test.drop('id', axis=1)

[ 0  0  0 ... 49 49 49]


In [388]:

# The age and race features
reloaded_model = tf.keras.models.load_model('Hyper_mixed_classification_with_all_features')
arr_rank_age_and_race = []
for i in range(len(id)):
    result = reloaded_model.predict(np.expand_dims(dataframe_test.loc[i],0))
    index_numpy_sorted = np.argsort(-result)
    temp_data = list(index_numpy_sorted[0])
    # print(temp_data.index(id[i]))
    # temp_data = index_numpy_sorted.where(id[i])
    # print(temp_data)
    arr_rank_age_and_race.append(temp_data.index(id[i]))

In [389]:
id_2 = dataframe_test_2['id'].values.tolist()
dataframe_test_2 = dataframe_test_2.drop('id', axis=1)

In [390]:
# The Non-features data
reloaded_model_2 = tf.keras.models.load_model('mixed_classification_for_masked')
arr_non_features = []
for j in range(len(id_2)):
    result = reloaded_model_2.predict(np.expand_dims(dataframe_test_2.loc[j],0))
    index_numpy_sorted = np.argsort(-result)
    temp_data = list(index_numpy_sorted[0])
    # print(temp_data.index(id[i]))
    # temp_data = index_numpy_sorted.where(id[i])
    # print(temp_data)
    arr_non_features.append(temp_data.index(id[j]))

In [391]:
print(arr_non_features)
print(arr_rank_age_and_race)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 6, 0, 40, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 5, 0, 1, 0, 4, 10, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [392]:
arr_result = []
zip_data = zip(arr_non_features,arr_rank_age_and_race)
# print(tuple(zip_data))
for list_1, list_2 in zip_data:
    arr_result.append(list_1 - list_2)
print(arr_result)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, -1, -1, -1, 0, 0, 0, -1, 3, 0, 1, 6, 0, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 0, 0, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 5, 0, 1, 0, 4, 8, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [393]:
w, p = wilcoxon(arr_result)
w, p

(2491.5, 3.1326565675887484e-05)